# Spark DataFrames

The Spark DataFrame is a feature that allows you to create and work with DataFrame objects. As you may have guessed, pandas inspired it.

Spark is well known for its ability to process large data sets. Spark DataFrames combine the scale and speed of Spark with the familiar query, filter, and analysis capabilities of pandas. Unlike pandas, which can only run on one computer, Spark can use distributed memory (and disk when necessary) to handle larger data sets and run computations more quickly.

Spark DataFrames allow us to modify and reuse our existing pandas code to scale up to much larger data sets. They also have better support for various data formats. We can even use a SQL interface to write distributed SQL queries that query large database systems and other data stores.

For this mission, we'll be working with a JSON file containing data from the 2010 U.S. Census. It has the following columns:

- age - Age (year)
- females - Number of females
- males - Number of males
- total - Total number of individuals
- year - Year column (2010 for all rows)

Let's open and explore the data set before we dive into Spark DataFrames.

Instructions

 1. Print the first four lines of census_2010.json.

In [1]:
# Find path to PySpark.
import findspark
findspark.init()

# Import PySpark and initialize SparkContext object.
import pyspark
sc = pyspark.SparkContext.getOrCreate()

In [2]:
rdd = sc.textFile('data/census_2010.json')
rdd.take(4)

['{"females": 1994141, "total": 4079669, "males": 2085528, "age": 0, "year": 2010}',
 '{"females": 1997991, "total": 4085341, "males": 2087350, "age": 1, "year": 2010}',
 '{"females": 2000746, "total": 4089295, "males": 2088549, "age": 2, "year": 2010}',
 '{"females": 2002756, "total": 4092221, "males": 2089465, "age": 3, "year": 2010}']

In previous missions, we explored reading data into an RDD object. Recall that an RDD is essentially a list of tuples with no enforced schema or structure of any kind. An RDD can have a variable number of elements in each tuple, and combinations of types between tuples.

RDDs are useful for representing unstructured data like text. Without them, we'd need to write a lot of custom Python code to interact with such data.

We use the SparkContext object to read data into an RDD:
<pre>
raw_data = sc.textFile(\"daily_show.tsv\")
daily_show = raw_data.map(lambda line: line.split('\t'))
</pre>
To use the familar DataFrame query interface from pandas, however, the data representation needs to include rows, columns, and types. Spark's implementation of DataFrames mirrors the pandas implementation, with logic for rows and columns.

The Spark SQL class is very powerful. It gives Spark more information about the data structure we're using and the computations we want to perform. Spark uses that information to optimize processes.

To take advantage of these features, we'll have to use the SQLContext object to structure external data as a DataFrame, instead of the SparkContext object.

We can query Spark DataFrame objects with SQL, which we'll explore in the next mission. The SQLContext class gets its name from this capability.

This class allows us to read in data and create new DataFrames from a wide range of sources. It can do this because it takes advantage of Spark's powerful Data Sources API.

File Formats

- JSON, CSV/TSV, XML
- Parquet, Amazon S3 (cloud storage service)

Big Data Systems

- Hive, Avro, HBase

SQL Database Systems
- MySQL, PostgreSQL

Data science organizations often use a wide range of systems to collect and store data, and they're constantly making changes to those systems. Spark DataFrames allow us to interface with different types of data, and ensure that our analysis logic will still work as the data storage mechanisms change.

Now that you've learned a bit about Spark DataFrames, let's read in census_2010.json. This data set contains valid JSON on each line, which is what Spark needs in order to read the data in properly.

In the following code cell, we:

1. Import SQLContext from the pyspark.sql library
2. Instantiate the SQLContext object (which requires the SparkContext object (sc) as a parameter), and assign it to the variable sqlCtx
3. Use the SQLContext method read.json() to read the JSON data set into a Spark DataFrame object named df
4. Print df's data type to confirm that we successfully read it in as a Spark DataFrame

In [4]:
# Import SQLContext
from pyspark.sql import SQLContext
# Pass in the SparkContext object `sc`
sqlCtx = SQLContext(sc)
# Read JSON data into a DataFrame object `df`
df = sqlCtx.read.json("data/census_2010.json")
# Print the type
print(type(df))

<class 'pyspark.sql.dataframe.DataFrame'>


When we read data into the SQLContext object, Spark:

Instantiates a Spark DataFrame object
Infers the schema from the data and associates it with the DataFrame
Reads in the data and distributes it across clusters (if multiple clusters are available)
Returns the DataFrame object
We expect the DataFrame Spark created to have the following columns, which were the keys in the JSON data set:
<pre>
age
females
males
total
year
</pre>
Spark has its own type system that's similar to the pandas type system. To create a DataFrame, Spark iterates over the data set twice - once to extract the structure of the columns, and once to infer each column's type. Let's use one of the Spark DataFrame instance methods to display the schema for the DataFrame we're working with.
1. Call the printSchema() method on the Spark DataFrame df to display the schema that Spark inferred

In [5]:
sqlCtx = SQLContext(sc)
df = sqlCtx.read.json("data/census_2010.json")
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- females: long (nullable = true)
 |-- males: long (nullable = true)
 |-- total: long (nullable = true)
 |-- year: long (nullable = true)



As we mentioned before, the pandas DataFrame heavily influenced the Spark DataFrame implementation. Here are some of the methods we can find in both:
<pre style='color:green'>
agg()
join()
sort()
where()
</pre>

Unlike pandas DataFrames, however, Spark DataFrames are immutable, which means we can't modify existing objects. Most transformations on an object return a new DataFrame reflecting the changes instead. As we discussed in previous missions, Spark's creators deliberately designed immutability into Spark to make it easier to work with distributed data structures.

Pandas and Spark DataFrames also have different underlying data structures. Pandas DataFrames are built around Series objects, while Spark DataFrames are built around RDDs. We can perform most of the same computations and transformations on Spark DataFrames that we can on pandas DataFrames, but the styles and methods are somewhat different. We'll explore how to perform common pandas functions with Spark in this mission.

Instructions

1. Use the show() method to print the first five rows of the DataFrame.

In [7]:
df.show(5)

+---+-------+-------+-------+----+
|age|females|  males|  total|year|
+---+-------+-------+-------+----+
|  0|1994141|2085528|4079669|2010|
|  1|1997991|2087350|4085341|2010|
|  2|2000746|2088549|4089295|2010|
|  3|2002756|2089465|4092221|2010|
|  4|2004366|2090436|4094802|2010|
+---+-------+-------+-------+----+
only showing top 5 rows



In pandas, we used the head() method to return the first n rows. This is one of the differences between the DataFrame implementations. Instead of returning a nicely formatted table of values, the head() method in Spark returns a list of row objects. Spark needs to return row objects for certain methods, such as head(), collect() and take().

You can access a row's attributes by the column name using dot notation, and by position using bracket notation with an index:
<pre style='color:green'>
row_one = df.head(5)[0]
# Access value for age
row_one.age
# Access the first value
row_one[0]
</pre>
Instructions

1. Use the head() method to return the first five rows in the DataFrame as a list of row objects, and assign the result to the variable first_five.
2. Print the age value for each row object in first_five.

In [20]:
first_five = df.head(5)
for obj in first_five:
    print(obj.age)

0
1
2
3
4


In pandas, we passed a string into a single pair of brackets ([]) to select an individual column, and passed in a list to select multiple columns:
<pre>
# Pandas DataFrame
df['age']
df[['age', 'males']]
</pre>
We can still use bracket notation in Spark. We'll need to pass in a list of string objects, though, even when we're only selecting one column.

Spark takes advantage of lazy loading with DataFrames, and will only display the results of an operation when we call the show() method. Instead of using bracket notation, we can also use the select() method to select columns:
<pre>
# Spark DataFrame
df.select('age')
df.select('age', 'males')
</pre>
In the following code cell, we demonstrate how to select and display the age column. Use what you've learned to take this a step farther and select multiple columns.

Instructions

1. Select the age, males, and females columns from the DataFrame and display them using the show() method.

In [25]:
#df[['age']].show()
df.select('age','males').show(5) # or df[['age','males',...]]

+---+-------+
|age|  males|
+---+-------+
|  0|2085528|
|  1|2087350|
|  2|2088549|
|  3|2089465|
|  4|2090436|
+---+-------+
only showing top 5 rows



In pandas, we used Boolean filtering to select only the rows we were interested in. Spark preserves the very same functionality and notation.

Instructions

1. Use the pandas notation for Boolean filtering to select the rows where age is greater than five.
2. Assign the resulting DataFrame to the variable five_plus.
3. Use the show() method to display five_plus.
## Duas maneiras de se fazer:

In [35]:
df.filter("Age > 5").show(5)

+---+-------+-------+-------+----+
|age|females|  males|  total|year|
+---+-------+-------+-------+----+
|  6|2007781|2093905|4101686|2010|
|  7|2010281|2097080|4107361|2010|
|  8|2013771|2101670|4115441|2010|
|  9|2018603|2108014|4126617|2010|
| 10|2023289|2114217|4137506|2010|
+---+-------+-------+-------+----+
only showing top 5 rows



In [37]:
five_plus = df[df['age'] > 5]
five_plus.show(5)

+---+-------+-------+-------+----+
|age|females|  males|  total|year|
+---+-------+-------+-------+----+
|  6|2007781|2093905|4101686|2010|
|  7|2010281|2097080|4107361|2010|
|  8|2013771|2101670|4115441|2010|
|  9|2018603|2108014|4126617|2010|
| 10|2023289|2114217|4137506|2010|
+---+-------+-------+-------+----+
only showing top 5 rows



We can compare the columns in Spark DataFrames with each other, and use the comparison criteria as a filter. For example, to get the rows where the population of males execeeded females in 2010, we'd write the same notation that we would use in pandas.

Instructions

1. Find all of the rows where females is less than males, and use show() to display the first 20 results.

In [38]:
df[df['females'] < df['males']].show(5)

+---+-------+-------+-------+----+
|age|females|  males|  total|year|
+---+-------+-------+-------+----+
|  0|1994141|2085528|4079669|2010|
|  1|1997991|2087350|4085341|2010|
|  2|2000746|2088549|4089295|2010|
|  3|2002756|2089465|4092221|2010|
|  4|2004366|2090436|4094802|2010|
+---+-------+-------+-------+----+
only showing top 5 rows



In [39]:
sc.stop()